In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import pandas as pd 
import numpy as np
import re

In [2]:
# read in the training data and define the X and Y values

import pandas as p
train=p.read_csv("./kaggle-sentiment/train.tsv", delimiter='\t')
y=train['Sentiment'].values
X=train['Phrase'].values

In [3]:
#split the data 60/40 and print to test that it worked
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

(93636,) (93636,) (62424,) (62424,)
almost in a class with that of Wilde
3
escape movie
2


In [4]:
# build the unigram TF-IDF vectorizer using english stopwords
tfidf_vectorizer = TfidfVectorizer(encoding='latin-1', use_idf=True, min_df=5, stop_words='english')

In [5]:
#Multinomial Naive Bayes and fit_train the data
MNB = MultinomialNB()

X1_train_vec = tfidf_vectorizer.fit_transform(X_train)

MNB.fit(X1_train_vec, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [6]:
#now test the data and show a confusion matrix
X1_test_vec = tfidf_vectorizer.transform(X_test)

# Run the model against the test set
X1_pred = MNB.predict(X1_test_vec)


target_names = ['0','1','2','3','4']


#had an issue with the confusion matrix and precision and recall scores here
#therefore I instituted a resolution found on github
X1_matrix = confusion_matrix(y_test, X1_pred)
print(X1_matrix)
print(classification_report(y_test, X1_pred, target_names = target_names))

[[  107  1144  1613    67     0]
 [   61  2580  7821   361     1]
 [   19  1168 28673  1987    17]
 [    0   147  7942  4883    96]
 [    0    11  1374  2164   188]]
             precision    recall  f1-score   support

          0       0.57      0.04      0.07      2931
          1       0.51      0.24      0.33     10824
          2       0.60      0.90      0.72     31864
          3       0.52      0.37      0.43     13068
          4       0.62      0.05      0.09      3737

avg / total       0.57      0.58      0.53     62424



In [7]:
# Print top 10 very positive and very negative words.
X1_neg = sorted(zip(MNB.coef_[0], tfidf_vectorizer.get_feature_names()))
X1_vneg = X1_neg[-10:]
print("top ten negative words: ")
print(X1_vneg)

X1_pos = sorted(zip(MNB.coef_[4], tfidf_vectorizer.get_feature_names()))
X1_vpos = X1_pos[-10:]
print("\r\n top ten positive words: ")
print(X1_vpos)

top ten negative words: 
[(-6.645979011120781, 'time'), (-6.62020923362483, 'does'), (-6.60369986228579, 'minutes'), (-6.517311233308192, 'dull'), (-6.355845020937053, 'just'), (-6.131355008437069, 'worst'), (-6.029810232070883, 'like'), (-5.792621365210278, 'film'), (-5.411480448479903, 'bad'), (-5.186697744581525, 'movie')]

 top ten positive words: 
[(-6.296148399433598, 'fun'), (-6.1660714027789965, 'entertaining'), (-6.123845477949642, 'performance'), (-6.038969240779541, 'performances'), (-5.946467095414271, 'great'), (-5.809197749822911, 'good'), (-5.652790662920261, 'funny'), (-5.485414582781967, 'movie'), (-5.444930328141257, 'best'), (-5.060214745410679, 'film')]


In [8]:
#Build the SVM
SVM = LinearSVC()

# Fit transform the training set with the tf_idf vectorizer
X1_train_svm = tfidf_vectorizer.fit_transform(X_train)

# Fit the model on the data 
SVM.fit(X1_train_svm, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [9]:
#Now we make the SVM model and print the confusion measures and classification report
X1_svm_test = tfidf_vectorizer.transform(X_test)

X1_svm_pred = SVM.predict(X1_svm_test)

X1_svm_matrix = confusion_matrix(y_test, X1_svm_pred)

print(X1_svm_matrix)
print(classification_report(y_test, X1_svm_pred, target_names = target_names))

[[  795  1387   624   117     8]
 [  589  4336  5245   629    25]
 [  163  2299 26557  2684   161]
 [   24   408  5604  6220   812]
 [    2    40   551  2010  1134]]
             precision    recall  f1-score   support

          0       0.51      0.27      0.35      2931
          1       0.51      0.40      0.45     10824
          2       0.69      0.83      0.75     31864
          3       0.53      0.48      0.50     13068
          4       0.53      0.30      0.39      3737

avg / total       0.61      0.63      0.61     62424



In [10]:
#print top 10 very positive and very negative words based on SVM

X1_svm_neg = sorted(zip(SVM.coef_[0], tfidf_vectorizer.get_feature_names()))
X1_vneg = X1_neg[-10:]
print("top ten negative words: ")
print(X1_vneg)

X1_pos = sorted(zip(SVM.coef_[4], tfidf_vectorizer.get_feature_names()))
X1_vpos = X1_pos[-10:]
print("\r\n top ten positive words: ")
print(X1_vpos)

top ten negative words: 
[(-6.645979011120781, 'time'), (-6.62020923362483, 'does'), (-6.60369986228579, 'minutes'), (-6.517311233308192, 'dull'), (-6.355845020937053, 'just'), (-6.131355008437069, 'worst'), (-6.029810232070883, 'like'), (-5.792621365210278, 'film'), (-5.411480448479903, 'bad'), (-5.186697744581525, 'movie')]

 top ten positive words: 
[(2.248662440844072, 'phenomenal'), (2.2841739494787334, 'masterpiece'), (2.3433897439404268, 'magnificent'), (2.3524538074188093, 'stunning'), (2.354474169030711, 'miraculous'), (2.3737024927478156, 'masterfully'), (2.3764047121600047, 'glorious'), (2.3812953006633313, 'masterful'), (2.390828846233555, 'zings'), (2.4682188984351496, 'perfection')]


In [11]:
#TASK 2 START
#create a bigram MNB using count vectorizer and show the confusion matrix and classification report
count_vec = CountVectorizer(encoding='latin-1', ngram_range=(1, 2), min_df=5, stop_words='english')
X2_train_mnb = count_vec.fit_transform(X_train)
MNB.fit(X2_train_mnb, y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [12]:
#transform using the vectorizer and compare it to the test set
X2_mnb_test = count_vec.transform(X_test)

X2_mnb_pred = MNB.predict(X2_mnb_test)

X2_mnb_matrix = confusion_matrix(y_test, X2_mnb_pred)

print(X2_mnb_matrix)
print(classification_report(y_test, X2_mnb_pred, target_names = target_names))

[[  867  1253   725    69    17]
 [  786  4440  4943   609    46]
 [  459  2961 24437  3600   407]
 [   41   513  5082  6375  1057]
 [    6    46   602  1911  1172]]
             precision    recall  f1-score   support

          0       0.40      0.30      0.34      2931
          1       0.48      0.41      0.44     10824
          2       0.68      0.77      0.72     31864
          3       0.51      0.49      0.50     13068
          4       0.43      0.31      0.36      3737

avg / total       0.58      0.60      0.59     62424



In [13]:
#top 10 positive and negative words
X2_mnb_neg = sorted(zip(MNB.coef_[0], count_vec.get_feature_names()))
X2_mnb_vneg = X2_mnb_neg[-10:]
print("top ten negative words: ")
print(X2_mnb_vneg)

X2_mnb_pos = sorted(zip(MNB.coef_[4], count_vec.get_feature_names()))
X2_mnb_vpos = X2_mnb_pos[-10:]
print("\r\n top ten positive words: ")
print(X2_mnb_vpos)

top ten negative words: 
[(-6.629714522576447, 'time'), (-6.61913241324591, 'characters'), (-6.608661113378615, 'minutes'), (-6.608661113378615, 'story'), (-6.598298326343068, 'comedy'), (-6.377218759249709, 'just'), (-5.825901774128982, 'like'), (-5.663620968218473, 'bad'), (-5.5205201245778, 'film'), (-5.0096945008118094, 'movie')]

 top ten positive words: 
[(-6.463080189370424, 'performance'), (-6.415452140381169, 'comedy'), (-6.369989766304411, 'great'), (-6.3407906116121495, 'story'), (-6.291657923034505, 'performances'), (-6.0177691727150595, 'good'), (-5.8771018432250886, 'funny'), (-5.782203101402293, 'best'), (-5.405967631609067, 'movie'), (-4.898537596844214, 'film')]


In [16]:
#transform and fit the SVM bigram model
X2_svm_train = count_vec.fit_transform(X_train)
SVM.fit(X2_svm_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [17]:
#transform and compare the model, then print a confusion matrix and classification report
X2_svm_test = count_vec.transform(X_test)

X2_svm_pred = SVM.predict(X2_svm_test)

X2_svm_matrix = confusion_matrix(y_test, X2_svm_pred)

print(X2_svm_matrix)
print(classification_report(y_test, X2_svm_pred, target_names = target_names))

[[ 1039  1276   542    63    11]
 [  864  4555  4911   457    37]
 [  252  2470 26246  2700   196]
 [   28   358  5383  6034  1265]
 [    5    27   452  1794  1459]]
             precision    recall  f1-score   support

          0       0.47      0.35      0.41      2931
          1       0.52      0.42      0.47     10824
          2       0.70      0.82      0.76     31864
          3       0.55      0.46      0.50     13068
          4       0.49      0.39      0.44      3737

avg / total       0.61      0.63      0.62     62424



In [18]:
#top 10 positive and negative words.
X2_svm_neg = sorted(zip(SVM.coef_[0], count_vec.get_feature_names()))
X2_svm_vneg = X2_svm_neg[-10:]
print("top ten negative words: ")
print(X2_svm_vneg)

X2_svm_pos = sorted(zip(SVM.coef_[4], count_vec.get_feature_names()))
X2_svm_vpos = X2_svm_pos[-10:]
print("\r\n top ten positive words: ")
print(X2_svm_vpos)

top ten negative words: 
[(1.7394373713229694, 'charm laughs'), (1.7467983581226574, 'unappealing'), (1.7584455220867992, 'unwatchable'), (1.7990447353153396, 'unbearable'), (1.8031740072361064, 'waste'), (1.8061705488504973, 'utterly incompetent'), (1.8574088594776197, 'disgusting'), (1.918246044592526, 'distasteful'), (1.959871590757011, 'pompous'), (1.9628017714568835, 'garbage')]

 top ten positive words: 
[(1.6517804964528011, 'masterful'), (1.6642263756689626, 'glorious'), (1.6946065546946185, 'flawless'), (1.73643958819818, 'masterfully'), (1.7382779029494584, 'gem'), (1.744519794707717, 'miraculous'), (1.8078519756398146, 'cut rest'), (1.8597827615419649, 'amazing'), (2.022840075411775, 'masterpiece'), (2.126909990638927, 'perfection')]


In [19]:
#TASK 3 BEGIN
#load the test data set
kaggle_test=p.read_csv("./kaggle-sentiment/train.tsv", delimiter='\t')
kaggle_train=p.read_csv("./kaggle-sentiment/train.tsv", delimiter='\t')

kag_y =kaggle_train['Sentiment'].values
kag_X =kaggle_train['Phrase'].values

kaggle_ids=kaggle_test['PhraseId'].values

kaggle_X_test=kaggle_test['Phrase'].values

X3_train = count_vec.fit_transform(kag_X)

X3_test=count_vec.transform(kaggle_X_test)

In [28]:
#predict the values
kaggle_pred=SVM.fit(X3_train, kag_y).predict(X3_test)

In [29]:
#test the cross validation measures of the scores
scores = cross_val_score(SVM, X3_train, kag_y, cv=5)

In [30]:
#print the scores
print(scores)
avg=sum(scores)/len(scores)
print(avg)

[0.57049495 0.54658166 0.5463138  0.54943928 0.56219161]
0.555004260716311


In [31]:
#create kaggle submission
kaggle_submission=zip(kaggle_ids, kaggle_pred)
outf=open('./kaggle-sentiment/kaggle_submission.csv', 'w')
outf.write('PhraseId,Sentiment\n')
for x, value in enumerate(kaggle_submission): outf.write(str(value[0]) + ',' + str(value[1]) + '\n')
outf.close()